In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-rimbwy29f75muq"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


#### Temasek Foundation Dataset

In [2]:
# First issue is that the training dataset is big, so we need to split it into smaller chunks and to something similar to SGD 
# So, subsample and fit, then figure out how to ensemble the results faster ... 


In [4]:
# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.load_data import prep_tf_node 

tf_meta_prompt, test_cases, tf_metric_map = prep_tf_node(prompt_mode=True)

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 
node = EvolNode(tf_meta_prompt, None, None, get_response=get_endpoint_response, test_cases=test_cases[:5],
                custom_metric_map=tf_metric_map) # setting manual test cases


In [5]:
node.evolve("i1", batch_size=5) # generation 1 || some sort of feedback ingestion is required || irrelevant function retrieved || redundant code inside
# Idea I. For functions which 'generate_prompt' main function does not depend on, remove them (Feature Request)

 

     :: Query time: 3.75s


Processing LLM queries: 100%|██████████| 5/5 [00:12<00:00,  2.45s/it]


 :: Total time elapsed: 12.24s, 0 errors
     :: Evolution time: 12.29s


Concurrent Execution of 25 Node functions to generate prompts ...:  80%|████████  | 20/25 [00:05<00:00,  7.27it/s]









  0%|          | 0.00/648 [00:00<?, ?iB/s]
  0%|          | 0.00/648 [00:00<?, ?iB/s]
Concurrent Execution of 25 Node functions to generate prompts ...:  88%|████████▊ | 22/25 [00:06<00:00,  3.49it/s]

Error downloading file: 404 Client Error: Not Found for url: https://example.com/grant_evaluation_guidelines.pdf
Error downloading file: 404 Client Error: Not Found for url: https://example.com/grant_evaluation_guidelines.pdf


...

Error downloading file: 404 Client Error: Not Found for url: https://example.com/grant_evaluation_guidelines.pdf
Error downloading file: 404 Client Error: Not Found for url: https://example.com/grant_evaluation_guidelines.pdf


Processing LLM queries: 100%|██████████| 45/45 [00:33<00:00,  1.34it/s]


 :: Total time elapsed: 33.69s, 0 errors

Code 4 outputs:
Test 0: {'decision': 'Yes', 'comment': "The project 'You are not alone - Human is near you' addresses the issue of malnutrition in Gujarat, India, by utilizing food waste to create nutritious meals for children. The project aligns with the challenge theme of Food Systems & Sustainable Agriculture and has the potential to improve the nutritional content of meals, combat malnutrition, and reduce food waste simultaneously. Although the project lacks specific objectives, proof of concept, and a proposed solution, its viability and potential impact are promising. The project also leverages the existing mid-day meal scheme, which provides a competitive advantage. Considering the project's alignment with the challenge theme, viability, potential impact, and competitive advantage, it is likely to be accepted."}
Test 1: {'decision': 'Yes', 'comment': "The project 'Marine Battery As A Service' has a strong alignment with the challenge the

Processing LLM queries: 100%|██████████| 28/28 [00:20<00:00,  1.33it/s]

 :: Total time elapsed: 20.99s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.67
  🎯 Functional fitness: 0.00
  ⭐ Global fitness:     0.33
  🔄 Compiled solutions:        5
  ⏱️ Time breakdown:
     :: Query time: 3.75s
     :: Evolution time: 12.29s
     :: Evaluation time: 62.04s
     :: Total time: 78.08s


📊 Code 0: Fitness: 33.3%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'project_description': 'Given the following details of a grant application, make a decision (Yes, No, Maybe) on whether to accept it or not, and a brief comment explanating your decision. \n    Assess how well the project aligns with the challenge theme, its viability, its potential impact, and any competitive advantage. \n    Consider the stage of development, the proof of concept status, and existing partnerships.\n    \n        \n    Grant Application Summary:\n\n    Grant Application Summary:\n    - **Project Theme:** Decarb

In [6]:
print(node.code)
# Firstly, while search google is used, get_celeb_age is not used here

from datetime import datetime
import http.client
import json
import os
from typing import Dict, Any


def _search_google(query: str) ->Dict[str, Any]:
    """
    Use Serper API to search Google for information
    
    Args:
        query (str): The search query
    
    Returns:
        Dict[str, Any]: Parsed JSON response from the API
    """
    conn = http.client.HTTPSConnection('google.serper.dev')
    payload = json.dumps({'q': query})
    headers = {'X-API-KEY': os.environ['SERPER_API_KEY'], 'Content-Type':
        'application/json'}
    try:
        conn.request('POST', '/search', payload, headers)
        res = conn.getresponse()
        data = res.read()
        return json.loads(data.decode('utf-8'))
    except Exception as e:
        print(f'Error occurred during API request: {str(e)}')
        return {}
    finally:
        conn.close()


def search_google(query: str) ->str:
    """ 
    Input query, return search result string from Google
    """
    result = _search_go